# 2022 Flatiron Machine Learning x Science Summer School

## Step 11: Train symbolic discriminator with extended library

Symbol pool:

* Input symbols: $x_0$, $x_1$

* Mathematical operators: $\times$, $\text{sin}$, $\text{cos}$

Let's explore some possible real functions.

In [2]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt
import joblib

import torch
import wandb

from srnet import SRNet, SRData
from sdnet import SDData
import srnet_utils as ut

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# load data
data_path = "data_1k"

in_var = "X00"
lat_var = "G00"
target_var = "F00"

mask_ext = ".mask"
masks = joblib.load(os.path.join(data_path, in_var + mask_ext))

train_data = SRData(data_path, in_var, lat_var, target_var, masks["train"])
val_data = SRData(data_path, in_var, lat_var, target_var, masks["val"])

In [4]:
x_data = train_data.in_data[:,0]
y_data = train_data.in_data[:,1]

In [5]:
fig, ax = plt.subplots()

ax.scatter(x_data, x_data**2)
ax.scatter(x_data, x_data**3)
ax.scatter(x_data, x_data**4)
ax.scatter(x_data, x_data**5)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
fig, ax = plt.subplots()

ax.scatter(x_data, np.sin(x_data))
#ax.scatter(x_data, np.sin(x_data**2))
ax.scatter(x_data, np.cos(x_data))
#ax.scatter(x_data, np.cos(x_data**2))

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
fig, ax = plt.subplots()

ax.scatter(x_data, x_data * np.sin(x_data))
ax.scatter(x_data, x_data * np.cos(x_data))
ax.scatter(x_data, x_data**3)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
_ = plt.figure()
ax = plt.axes(projection='3d')

ax.scatter3D(x_data, y_data, x_data*y_data)
ax.scatter3D(x_data, y_data, x_data**2*y_data)
# ax.scatter3D(x_data, y_data, x_data*np.cos(y_data))

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Step 11.2: Train bottleneck masked DSN with SD regularization on `F00_v3`

Extension of the SD library to version 3:

```
X00[:,0]**2
X00[:,1]**2
X00[:,0]**3
X00[:,1]**3
np.sin(X00[:,0])
np.sin(X00[:,1])
np.cos(X00[:,0])
np.cos(X00[:,1])
X00[:,0] * X00[:,1]
```

In [9]:
# set wandb project
wandb_project = "112-bn-mask-DSN-sd-study-F00_v3"

In [10]:
# hyperparams = {
#     "arch": {
#         "in_size": train_data.in_data.shape[1],
#         "out_size": train_data.target_data.shape[1],
#         "hid_num": (2,0),
#         "hid_size": 32, 
#         "hid_type": ("DSN", "MLP"),
#         "hid_kwargs": {
#             "alpha": [[1,0],[0,1],[1,1]],
#             "norm": None,
#             "prune": None,
#             },
#         "lat_size": 3,
#         },
#     "epochs": 30000,
#     "runtime": None,
#     "batch_size": train_data.in_data.shape[0],
#     "shuffle": False,
#     "lr": 1e-4,
#     "wd": 1e-4,
#     "l1": 0.0,
#     "a1": 0.0,
#     "a2": 0.0,
#     "e1": 0.0,
#     "e2": 0.0,
#     "gc": 0.0,
#     "sd": 1e-4,
#     "disc": {
#         "hid_num": 2,
#         "hid_size": 128,
#         "lr": 1e-4,
#         "wd": 1e-4,
#         "iters": 5,
#         "gp": 1e-4,
#     },
# }

In [11]:
# define hyperparameter study
hp_study = {
    "method": "random",
    "parameters": {
        "sd": {
            "values": [1e-7, 1e-6, 1e-5, 1e-4]
        },
        "disc": {
            "parameters": {
                "hid_num": {
                    "values": [4, 6, 8]
                },
                "hid_size": {
                    "values": [128, 256, 512]
                },
                "lr": {
                    "values": [1e-5, 1e-4, 1e-3, 1e-2]
                },
                "iters": {
                    "values": [1, 5, 10]
                },
                "gp": {
                    "values": [1e-5, 1e-4, 1e-3]
                },
            }
        }
    }
}

In [ ]:
# create sweep
sweep_id = wandb.sweep(hp_study, project=wandb_project)

<img src="results/112-bn-mask-DSN-sd-study-F00_v3.png">

With the increased real function library we are not able to learn high correlation latent features.

What are possible options to progress?

* Embed additional information (input data, derivative, curvature)

* Train for more epochs

* Increase input data range

* Resample input data

* Resample coefficients of library functions

* Restart from trained DSN

* Activate SD regularization only after X epochs

* Select library functions depending on the number of input features

* Train with GhostAdam

General next steps:

* Input data "noise"

* Resolve bottleneck

* Input data dimension

* Change complexity of $f(x)$